# Data exploration #1

### In this notebook we will perform a preliminary statistical exploration of the dataset in order to get a better sense of the problem's formulation and any outliers or extreme scenarios that we should take care.

In [192]:
import datetime
import numpy as np
import pandas as pd
import os
import sys
import plotly.plotly as py
from plotly import tools
import numpy as np
import plotly.figure_factory as ff
import seaborn as sns
from scipy import stats
import numpy as np
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
import tensorflow as tf

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)



ImportError: cannot import name 'dataProc' from 'src.processing' (/Users/noresources/Pycharm_projects/justbeatit/src/processing.py)

In [184]:
dataset = pd.read_csv(
    '/Users/noresources/Pycharm_projects/justbeatit/data/routes.csv', sep='	', low_memory=False)

In [185]:
# Quick peek at the dataset
dataset

,passenger_id,source_latitude,source_longitude,source_address,destination_latitude,destination_longitude,destination_address,request_date
0,41037,-12.088156,-77.016065,"Avenida Nicolás de Arriola 314, La Victoria 13",-12.108531,-77.044891,"Calle Carlos Graña Elisande 340, San Isidro 27",2015-09-01 00:00:04
1,116591,-12.099957,-77.036497,"Av Los Conquistadores 392, San Isidro 15073",-12.119686,-76.999969,"Bruselas 228, La Calera De La Merced",2015-09-01 00:00:15
2,86426,-12.099153,-77.019425,"Av. República de Panamá 3537, San Isidro 27",-12.076505,-77.089305,"Av. La Marina cdra. 25, San Miguel 32",2015-09-01 00:00:17
3,53610,-12.110271,-77.028945,"Junín 225, Miraflores",-12.132221,-77.027021,"Calle San Fernando 380, Miraflores 18",2015-09-01 00:00:29
4,102927,-12.098430,-77.026246,"Av. República De Colombia 791, San Isidro",-12.099529,-76.990486,"Calle Mozart 201, San Borja 41",2015-09-01 00:00:31
5,163806,-12.142475,-77.011614,"Jr. Rivera, Surco",-12.083100,-77.052130,"Avenida San Felipe, Jesús María 11",2015-09-01 00:00:34
6,96702,-12.106040,-77.038510,Pardo Y Aliaga 652,-12.092780,-77.062653,"Avenida Javier Prado Oeste 654, Magdalena del ...",2015-09-01 00:00:50
7,173422,-12.088830,-76.989520,"Cavallini 261, Lima 15037",-12.133783,-77.010262,"Carlos Baca Flor 646, Lima 15048",2015-09-01 00:00:54
8,116591,-12.099957,-77.036497,"Av Los Conquistadores 392, San Isidro 15073",-12.119686,-76.999969,"Bruselas 228, La Calera De La Merced",2015-09-01 00:00:57
9,226380,-12.076007,-77.037887,"Avenida Francisco Javier Mariategui 183, Jesús...",-12.089933,-77.052312,"Av. Salaverry 2370, Jesús María 00011",2015-09-01 00:00:58


### Lets perform some dataset transformations:

In [186]:
# Lets fix the date into a datetime:
dataset['request_date'] = pd.to_datetime(dataset['request_date'], format='%Y-%m-%d %H:%M:%S')
# routes_asll['week_day'] = routes_all.apply(lambda x: x['request_date'].weekday(), axis=1)

# 1 ride == 1 volume unit
dataset['ride_volume'] = 1

### Next, lets extract some series of variable granularity to get a sense of the std and the mean of the volume per week, day, hour and 15 mins.

In [189]:
# quart_sum = [0]
# hour_sum = [0]
# day_sum = [0]
# week_sum = [0]

# start = routes_all['request_date'][0]
# quart, hour, day, week = (start.minute // 15, start.hour, start.weekday(), start.day // 7)

# for date in routes_all['request_date']:

#     # Quarts
#     if date.minute // 15 == quart:
#         quart_sum[-1] += 1
#     else:
#         quart_sum.append(1)
#         quart = date.minute // 15

#     # Hours
#     if date.hour == hour:
#         hour_sum[-1] += 1
#     else:
#         hour_sum.append(1)
#         hour = date.hour
#     # Days
#     if date.day == day:
#         day_sum[-1] += 1
#     else:
#         day_sum.append(1)
#         day = date.day

#     # Weeks
#     if date.day // 7 == week:
#         week_sum[-1] += 1

#     else:
#         week_sum.append(1)
#         week = day // 7



quart_sum = dataProc.aggregate_by_mins(dataset=dataset, datetime_column='request_date', minutes=15 )['ride_volume']

hour_sum = dataProc.aggregate_by_hours(dataset=dataset, datetime_column='request_date', hours= 1)['ride_volume']

day_sum = dataProc.aggregate_by_hours(dataset=dataset, datetime_column='request_date', hours= 24)['ride_volume']

week_sum = dataProc.aggregate_by_hours(dataset=dataset, datetime_column='request_date', hours= 24 * 7)['ride_volume']




ImportError: cannot import name 'dataProc' from 'src.processing' (/Users/noresources/Pycharm_projects/justbeatit/src/processing.py)

# Now that we have the sums for a gradient granularity, we can start checking for outliers:

In [150]:
def plot_outlier_boxes(data_array):
    # Create distplot with custom bin_size
    trace0 = go.Box(
        y=data_array,
        name="All Points",
        jitter=0.3,
        pointpos=-1.8,
        boxpoints='all',
        marker=dict(
            color='rgb(7,40,89)'),
        line=dict(
            color='rgb(7,40,89)')
    )

    trace1 = go.Box(
        y=data_array,
        name="Only Whiskers",
        boxpoints=False,
        marker=dict(
            color='rgb(9,56,125)'),
        line=dict(
            color='rgb(9,56,125)')
    )

    trace2 = go.Box(
        y=data_array,
        name="Suspected Outliers",
        boxpoints='suspectedoutliers',
        marker=dict(
            color='rgb(8,81,156)',
            outliercolor='rgba(219, 64, 82, 0.6)',
            line=dict(
                outliercolor='rgba(219, 64, 82, 0.6)',
                outlierwidth=2)),
        line=dict(
            color='rgb(8,81,156)')
    )

    trace3 = go.Box(
        y=data_array,
        name="Whiskers and Outliers",
        boxpoints='outliers',
        marker=dict(
            color='rgb(107,174,214)'),
        line=dict(
            color='rgb(107,174,214)')
    )

    data = [trace0, trace1, trace2, trace3]

    layout = go.Layout(
        title="Box Plot Styling Outliers"
    )

    fig = go.Figure(data=data, layout=layout)
    return fig

# Checking per hour outliers
fig = plot_outlier_boxes(hour_sum)
iplot(fig, filename="Box Plot Styling Outliers")


/Users/noresources/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [102]:
# Plotting the histogram per hour for verification
data = [go.Histogram(x=hour_sum)]
py.iplot(data, filename='Day histogram')

## From the graphs above it is clear that the aggregation produces outliers that must be removed. We are going to use the z score to do that:

In [175]:
threshold = 2
z = np.abs(stats.zscore(hour_sum))
outliers = list(np.where(z > threshold)[0])
for index in sorted(outliers, reverse=True):
    del hour_sum[index]



In [176]:
# Checking per hour outliers
fig = plot_outlier_boxes(hour_sum)
py.iplot(fig, filename="Box Plot Styling Outliers")


/Users/noresources/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## We have successfully trimmed the outliers.

### First, let's get some basic stats out of the dataset:

In [135]:
print('Number of rows:', len(routes_all))

days_counter = np.zeros(7)
hours_counter = np.zeros(24)
quarters_counter = np.zeros(24*4)
five_counter = np.zeros(12)

for row in routes_all['request_date']:
    
    # Grab the date time as lists
    date, time = row.split(sep=' ')
    date = list(map(int, date.split(sep='-')))
    time = list(map(int, time.split(sep=':')))
    
    # Import them into a datetime var
    dtt = datetime.datetime(
        date[0], date[1], date[2], time[0], time[1], time[2])
    
    # Update the counters
    days_counter[dtt.weekday()] += 1
    hours_counter[time[0]] += 1
    quarters_counter[(time[1]//15) + (time[0] * 4)] += 1
    five_counter[time[1]//5] += 1
    
    
    
quartile_sums = np.zeros(4)

for idx, quart in enumerate(quarters_counter):
    quartile_sums[idx % 4] += quart


print('Days:', days_counter)
print('Hours:', hours_counter)
print('Quarts:',quartile_sums)
for i in range(0, 24):
    print(quarters_counter[i*4:i*4+4])

Number of rows: 2045260


AttributeError: 'Timestamp' object has no attribute 'split'

In [ ]:
def grab_dates_in_range(start=datetime.datetime, end=datetime.datetime)
    relevant = routes_all[routes_all['request_date']]
    


### Now that we have some basic counters of variable granularity: